# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
#Copying csv path
file_to_load = "../Resources/clean_city_data.csv"

#Loading csv path onto dataframe using pandas read function
city_df = pd.read_csv(file_to_load)

city_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)
humidity = city_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_df[["Latitude", "Longitude"]]

In [ ]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_city_df = city_df.loc[(city_df["Max Temperature"] > 70) & (city_df["Max Temperature"] < 80) & 
            (city_df["Cloudiness"] == 0) & (city_df["Wind Speed"] < 10), :]

In [6]:
new_city_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Gobabis,-22.45,18.97,73.40,16,0,5.82,NaN,1595861577
20,Erenhot,43.65,111.98,78.17,39,0,4.52,CN,1595861600
29,Yangcun,39.36,117.06,75.20,88,0,3.47,CN,1595861610
230,Henties Bay,-22.12,14.28,73.00,52,0,7.00,NaN,1595861868
235,Lagoa,39.05,-27.98,73.11,86,0,6.22,PT,1595861761
256,Ponta do Sol,32.67,-17.10,75.99,65,0,5.17,PT,1595861750
465,Mwinilunga,-11.74,24.43,77.23,21,0,7.63,ZM,1595862214
476,Xapuri,-10.65,-68.50,75.20,73,0,4.70,BR,1595862231
482,Zambezi,-13.54,23.10,79.63,18,0,8.81,ZM,1595862239
518,Fuling,29.70,107.39,77.04,85,0,1.90,CN,1595862280


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#Empty list to store Hotel data.
Hotel = []

#Get latitude and longitude from dataframe (row/column) using 'iloc'.
for i in range(0,len(new_city_df)):
   
    latitude = new_city_df.iloc[i,1]
    longitude = new_city_df.iloc[i,2]
    
    #Setting necessary parameters to search for.
    params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
    }
    
    #Assigning location parameter to latitude, longitude.
    params["location"] = f"{latitude},{longitude}"
    
    #Requesting Places API to retreive Hotel info.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places_requested = requests.get(url, params=params).json()
    try:
        Hotel.append(places_requested['results'][0]['name'])
    
    except IndexError:
        hotel_name.append(np.nan)

In [25]:
new_city_df["Hotel Name"] = Hotel
hotel_df = new_city_df.dropna(how='any')

hotel_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,Erenhot,43.65,111.98,78.17,39,0,4.52,CN,1595861600,Moli International Hotel
29,Yangcun,39.36,117.06,75.20,88,0,3.47,CN,1595861610,Holiday Inn Tianjin Wuqing
235,Lagoa,39.05,-27.98,73.11,86,0,6.22,PT,1595861761,Graciosa Resort - Biosphere Island Hotel
256,Ponta do Sol,32.67,-17.10,75.99,65,0,5.17,PT,1595861750,Estalagem Da Ponta Do Sol
465,Mwinilunga,-11.74,24.43,77.23,21,0,7.63,ZM,1595862214,Muzina Guest House


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))